# Trace Through of using the Emulator API

In [36]:
from pypokerengine.api.emulator import Emulator
import numpy as np
from raise_player import RaisedPlayer
from PPOplayer import PPOPlayer

In [37]:
emulator = Emulator()
raise_player_1 = RaisedPlayer()
p1_uuid = "player1"

ppo_player_2 = RaisedPlayer()
ppoplayer_uuid = "player2"

In [38]:
emulator.set_game_rule(player_num=2, max_round=1, small_blind_amount=10, ante_amount=0)
emulator.register_player(p1_uuid, raise_player_1)
emulator.register_player(ppoplayer_uuid, ppo_player_2)

In [39]:
players_info = {
            p1_uuid: {"stack": 1000, "name": "player1"},
            ppoplayer_uuid: {"stack": 1000, "name": "ppo_player"},
        }

In [40]:
init_state = emulator.generate_initial_game_state(players_info)
print(init_state)

{'round_count': 0, 'small_blind_amount': 10, 'street': 0, 'next_player': None, 'table': <pypokerengine.engine.table.Table object at 0x159705b80>}


In [42]:
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'] # These are straightforward
SUITS = ['S', 'H', 'D', 'C'] # Spades, Hearts, Diamonds, Clubs

def state_to_observation(hole_card, round_state, valid_actions):
        # Helper function to convert cards to one-hot encoding
        def one_hot_52(cards):
            vec = np.zeros(52, dtype=np.float32)
            # Handle None case
            if cards is None:
                return vec
            for c in cards:
                if not isinstance(c, str) or len(c) != 2:
                    continue  # skip cards with issues
                suit = c[0].upper()
                rank = c[1].upper()
                if rank in RANKS and suit in SUITS:
                    idx = SUITS.index(suit) * 13 + RANKS.index(rank)
                    vec[idx] = 1.0
            return vec

        # Extract game state information
        community = round_state["community_card"]
        next_player_pos = round_state["next_player"]
        
        # Determine who the current player is (agent) and opponent
        agent_seat = round_state["seats"][next_player_pos]
        agent_uuid = agent_seat["uuid"]
        
        # Find opponent seat
        opponent_seat = next(s for s in round_state["seats"] if s["uuid"] != agent_uuid)
        
        # Extract stack sizes
        agent_stack = agent_seat["stack"]
        opponent_stack = opponent_seat["stack"]
    
        
        street = round_state["street"].lower()
        
        # Extract pot size, assuming that there is no side 
        pot_amount = round_state["pot"]["main"]["amount"]
        
        # Determine if current player is small blind
        sb_pos = round_state.get("small_blind_pos")
        sb_flag = 1.0 if next_player_pos == sb_pos else 0.0
        
        # Encode the street (one-hot)
        street_vec = np.zeros(4, dtype=np.float32)
        street_idx = {"preflop": 0, "flop": 1, "turn": 2, "river": 3}.get(street, 0)
        street_vec[street_idx] = 1.0
        
        def to_call_flag(agent_uuid: str, rs: dict) -> float:
            street = rs["street"].lower()
            hist   = rs["action_histories"][street]
            current_bet = max((h["amount"] for h in hist), default=0)
            your_bet    = max((h["amount"] for h in hist if h["uuid"] == agent_uuid),
                          default=0)
            return 1.0 if current_bet > your_bet else 0.0

        call_cost_flag = to_call_flag(agent_uuid, round_state)  # 0 or 1

        can_raise = 1.0 if any(d['action'] == 'raise' for d in valid_actions) else 0.0

        initial_stack = 1000

        obs = np.concatenate([
            one_hot_52(hole_card),                  # 0-51
            one_hot_52(community),                  # 52-103
            np.array([
                agent_stack   / initial_stack, # 104
                opponent_stack/ initial_stack, # 105
                pot_amount    / (2*initial_stack),  # 106
                sb_flag                                   # 107
            ], dtype=np.float32),
            street_vec,                             # 108-111
            np.array([
                can_raise,          # 112
                call_cost_flag      # 113  ← NEW bit
            ], dtype=np.float32)
        ])
        
        return obs.astype(np.float32)

In [43]:
new_state, events = emulator.start_new_round(init_state)
print(f"New state: {new_state} \n")
print(f"Num Events: {len(events)} \n")
print(f"1st Event: {events[0]} \n")
print(f"2nd Event: {events[1]} \n")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 0, 'next_player': 1, 'table': <pypokerengine.engine.table.Table object at 0x159705880>} 

Num Events: 2 

1st Event: {'type': 'event_new_street', 'street': 'preflop', 'round_state': {'street': 'preflop', 'pot': {'main': {'amount': 30}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 980, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 990, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}]}}} 

2nd Event: {'type': 'event_ask_player', 'uuid': 'player2', 'valid_actions': [{'action': 'fold'}, {'action': 'call'}, {'action': 'raise'}], 'round_state': {'street': 'preflop', 'p

In [44]:
round_state = events[1]["round_state"]
print(f"round state: {round_state}")
curr_player_uuid = events[1]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(new_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[1]["valid_actions"]
print(f"valid actions: {valid_actions}")

round state: {'street': 'preflop', 'pot': {'main': {'amount': 30}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 980, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 990, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}]}}
Next player uuid: player2
hole_cards: ['H6', 'HA']
valid actions: [{'action': 'fold'}, {'action': 'call'}, {'action': 'raise'}]


In [45]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.99 , 0.98 , 0.015, 1.   ,
       1.   , 0.   , 0.   , 0.   , 1.   , 1.   ], dtype=float32)

In [46]:
curr_player = emulator.fetch_player(curr_player_uuid)
action = curr_player.declare_action(valid_actions, hole_cards, round_state)
print(action)

raise


In [47]:
updated_state, events = emulator.apply_action(new_state, action)

In [48]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")
print()

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 0, 'next_player': 0, 'table': <pypokerengine.engine.table.Table object at 0x159706210>} 

Events: [{'type': 'event_ask_player', 'uuid': 'player1', 'valid_actions': [{'action': 'fold'}, {'action': 'call'}, {'action': 'raise'}], 'round_state': {'street': 'preflop', 'pot': {'main': {'amount': 60}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 0, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 980, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 960, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}]}}}] 




In [49]:
round_state = events[0]["round_state"]
print(f"round state: {round_state}")
curr_player_uuid = events[0]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(updated_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[0]["valid_actions"]
print(f"valid actions: {valid_actions}")

round state: {'street': 'preflop', 'pot': {'main': {'amount': 60}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 0, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 980, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 960, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}]}}
Next player uuid: player1
hole_cards: ['S6', 'CQ']
valid actions: [{'action': 'fold'}, {'action': 'call'}, {'action': 'raise'}]


In [50]:
curr_player = emulator.fetch_player(curr_player_uuid)
action = curr_player.declare_action(valid_actions, hole_cards, round_state)
print(action)

raise


In [51]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.96, 0.03, 0.  , 1.  , 0.  ,
       0.  , 0.  , 1.  , 1.  ], dtype=float32)

In [52]:
updated_state, events = emulator.apply_action(updated_state, "call")

In [53]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 1, 'next_player': 1, 'table': <pypokerengine.engine.table.Table object at 0x15978a5d0>} 

Events: [{'type': 'event_new_street', 'street': 'flop', 'round_state': {'street': 'flop', 'pot': {'main': {'amount': 80}, 'side': []}, 'community_card': ['H9', 'S5', 'C5'], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 960, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 960, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'CALL', 'amount': 40, 'paid': 20, 'uuid': 'player1'}], 'flop': []}}}, {'type': 'event_ask_player

In [55]:
round_state = events[1]["round_state"]
print(f"round state: {round_state}")
curr_player_uuid = events[1]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(updated_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[1]["valid_actions"]
print(f"valid actions: {valid_actions}")

round state: {'street': 'flop', 'pot': {'main': {'amount': 80}, 'side': []}, 'community_card': ['H9', 'S5', 'C5'], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 960, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 960, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'CALL', 'amount': 40, 'paid': 20, 'uuid': 'player1'}], 'flop': []}}
Next player uuid: player2
hole_cards: ['H6', 'HA']
valid actions: [{'action': 'fold'}, {'action': 'call'}, {'action': 'raise'}]


In [56]:
curr_player = emulator.fetch_player(curr_player_uuid)
action = curr_player.declare_action(valid_actions, hole_cards, round_state)
print(action)

raise


In [57]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.96, 0.96, 0.04, 1.  , 0.  , 1.  ,
       0.  , 0.  , 1.  , 0.  ], dtype=float32)

In [21]:
updated_state, events = emulator.apply_action(updated_state, action)

In [22]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 0, 'next_player': 0, 'table': <pypokerengine.engine.table.Table object at 0x1596b9370>} 

Events: [{'type': 'event_ask_player', 'uuid': 'player1', 'valid_actions': [{'action': 'fold'}, {'action': 'call'}], 'round_state': {'street': 'preflop', 'pot': {'main': {'amount': 140}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 0, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 940, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 920, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'RAISE', 'amount': 60, 'paid': 40, 'add_amount':

In [23]:
round_state = events[0]["round_state"]
print(f"round state: {round_state}")
curr_player_uuid = events[0]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(updated_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[0]["valid_actions"]
print(f"valid actions: {valid_actions}")

round state: {'street': 'preflop', 'pot': {'main': {'amount': 140}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 0, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 940, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 920, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'RAISE', 'amount': 60, 'paid': 40, 'add_amount': 20, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 80, 'paid': 40, 'add_amount': 20, 'uuid': 'player2'}]}}
Next player uuid: player1
hole_cards: ['DT', 'DQ']
valid actions: [{'action': 'fold'}, {'action': 'call'}]


In [24]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.94, 0.92, 0.07, 0.  , 1.  , 0.  ,
       0.  , 0.  , 0.  ], dtype=float32)

In [25]:
updated_state, events = emulator.apply_action(updated_state, action)

In [26]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")

round_state = events[0]["round_state"]
print(f"round state: {round_state}")
curr_player_uuid = events[0]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(updated_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[0]["valid_actions"]
print(f"valid actions: {valid_actions}")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 0, 'next_player': 1, 'table': <pypokerengine.engine.table.Table object at 0x1596b8bc0>} 

Events: [{'type': 'event_ask_player', 'uuid': 'player2', 'valid_actions': [{'action': 'fold'}, {'action': 'call'}], 'round_state': {'street': 'preflop', 'pot': {'main': {'amount': 180}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 900, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 920, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'RAISE', 'amount': 60, 'paid': 40, 'add_amount':

In [27]:
curr_player = emulator.fetch_player(curr_player_uuid)
action = curr_player.declare_action(valid_actions, hole_cards, round_state)
print(action)

call


In [28]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.92, 0.9 , 0.09, 1.  , 1.  , 0.  ,
       0.  , 0.  , 0.  ], dtype=float32)

In [29]:
updated_state, events = emulator.apply_action(updated_state, action)

In [30]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")

round_state = events[1]["round_state"]
print(f"round state: {round_state}") 
curr_player_uuid = events[1]["uuid"]
print(f"Next player uuid: {curr_player_uuid}")
hole_cards = emulator.get_hole_cards(updated_state)
print(f"hole_cards: {hole_cards}")
valid_actions = events[1]["valid_actions"]
print(f"valid actions: {valid_actions}")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 1, 'next_player': 1, 'table': <pypokerengine.engine.table.Table object at 0x1596b9be0>} 

Events: [{'type': 'event_new_street', 'street': 'flop', 'round_state': {'street': 'flop', 'pot': {'main': {'amount': 200}, 'side': []}, 'community_card': ['HK', 'C3', 'SJ'], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 900, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 900, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'RAISE', 'amount': 60, 'paid': 40, 'add_amount': 20, 'uuid': 'player1'}, {'action': 'RAISE', 'a

In [31]:
curr_player = emulator.fetch_player(curr_player_uuid)
action = curr_player.declare_action(valid_actions, hole_cards, round_state)
print(action)

raise


In [32]:
state_to_observation(hole_cards, round_state, valid_actions)

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0.9, 0.9, 0.1, 1. , 0. , 1. , 0. , 0. , 1. ], dtype=float32)

In [33]:
updated_state, events = emulator.apply_action(updated_state, "fold")

In [34]:
print(f"New state: {updated_state} \n")
print(f"Events: {events} \n")
print(f"len of events: {len(events)}")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 5, 'next_player': 0, 'table': <pypokerengine.engine.table.Table object at 0x1596b8530>} 

Events: [{'type': 'event_round_finish', 'round_state': {'street': 'showdown', 'pot': {'main': {'amount': 200}, 'side': []}, 'community_card': ['HK', 'C3', 'SJ', 'C5', 'D7'], 'dealer_btn': 0, 'next_player': 0, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 10, 'seats': [{'name': 'player1', 'uuid': 'player1', 'stack': 1100, 'state': 'participating'}, {'name': 'ppo_player', 'uuid': 'player2', 'stack': 900, 'state': 'folded'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 10, 'add_amount': 10, 'uuid': 'player2'}, {'action': 'BIGBLIND', 'amount': 20, 'add_amount': 10, 'uuid': 'player1'}, {'action': 'RAISE', 'amount': 40, 'paid': 30, 'add_amount': 20, 'uuid': 'player2'}, {'action': 'RAISE', 'amount': 60, 'paid': 40, 'add_amount': 20, 'uuid': 'player1'}, {'action': 'RAISE', 'amount'

In [35]:
new_state, events = emulator.start_new_round(updated_state)
print(f"New state: {new_state} \n")
print(f"Num Events: {len(events)} \n")
print(f"1st Event: {events[0]} \n")
print(f"2nd Event: {events[1]} \n")

New state: {'round_count': 1, 'small_blind_amount': 10, 'street': 5, 'next_player': 0, 'table': <pypokerengine.engine.table.Table object at 0x1596b8260>} 

Num Events: 1 

1st Event: {'type': 'event_game_finish', 'players': [{'uuid': 'player1', 'stack': 1100}, {'uuid': 'player2', 'stack': 900}]} 



IndexError: list index out of range